<a href="https://colab.research.google.com/github/hasiburrahman1/NLP/blob/master/complex_type_QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np 
import pandas as pd 
import os
import re
import matplotlib.pyplot as plt
import xml.etree.ElementTree as et 
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, SimpleRNN, GRU, LSTM, Bidirectional, Dropout, Input, Conv2D, MaxPool2D
from keras.layers.embeddings import Embedding
from keras.initializers import Constant
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras.layers import Reshape, Flatten, Dropout, Concatenate
from keras.optimizers import Adam
from keras.models import Model
from keras.layers.convolutional import Conv1D
from keras.layers import Conv1D, Dense, MaxPool1D, Flatten, Input, GlobalMaxPooling1D

Using TensorFlow backend.


In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
import tensorflow as tf
import collections
import unicodedata
import re
import numpy as np
import os
import io
import time
import xlrd
import csv

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
'''
cols_to_use = ['Question', 'Answer'] # or [0,1,2,3]
df = pd.read_csv('data.csv', usecols= cols_to_use)
'''

"\ncols_to_use = ['Question', 'Answer'] # or [0,1,2,3]\ndf = pd.read_csv('data.csv', usecols= cols_to_use)\n"

In [6]:
'''
mport pandas
df = pandas.read_excel(r"/test.xlsx")
df = df.drop('label', 1)
#print the column names
print(df.columns)

#get the values for a given column
#values = df['collumn_name'].values

#get a data frame with selected columns
FORMAT = ['Question', 'Answer']
df_selected = df[FORMAT]


df.to_csv('dataset_csv.csv')
!cp dataset_csv.csv "drive/My Drive/"

df.describe()
'''

'\nmport pandas\ndf = pandas.read_excel(r"/test.xlsx")\ndf = df.drop(\'label\', 1)\n#print the column names\nprint(df.columns)\n\n#get the values for a given column\n#values = df[\'collumn_name\'].values\n\n#get a data frame with selected columns\nFORMAT = [\'Question\', \'Answer\']\ndf_selected = df[FORMAT]\n\n\ndf.to_csv(\'dataset_csv.csv\')\n!cp dataset_csv.csv "drive/My Drive/"\n\ndf.describe()\n'

---------------------------**Start**-------------------------------------------------------------------------------------------------------------------------------------

In [7]:
# dataset path
#path_to_file = "/datasetCleaned.txt"

path_to_file = "/content/drive/My Drive/Colab Notebooks/dataset/QA_complex_type.txt"
#path_to_file = "/content/drive/My Drive/Colab Notebooks/dataset/test_Text_Document.txt"

In [8]:
def preprocess_sentence(w):
#     remove punctuations from lines
    w = re.sub(r"([?.!,¿।])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    
    w = w.rstrip().strip()
    #print(w)
    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    w = '<start> ' + w + ' <end>'
    #print(w)
    return w

In [9]:
def vocabularyCounter(dataQuestion, dataAnswer):
    # setup Vocabulary 
    question_words = collections.Counter([word for sentence in dataQuestion for word in sentence.split()])
    answer_words = collections.Counter([word for sentence in dataAnswer for word in sentence.split()])
    
    print('Total English words: {}'.format(len([word for sentence in dataQuestion for word in sentence.split()])))
    print('Unique English words: {}'.format(len(question_words)))
    print('Most Common Words: "' + '" "'.join(list(zip(*question_words.most_common(10)))[0]) + '"\n')

    print('Total Bangla words: {}'.format(len([word for sentence in dataAnswer for word in sentence.split()])))
    print('Unique Bangla words: {}'.format(len(answer_words)))
    print('Most Common Words: "' + '" "'.join(list(zip(*answer_words.most_common(10)))[0]) + '"')

In [10]:

# 1. Remove the accents
# 2. Clean the sentences
# 3. Return word pairs in the format: [ENGLISH, BANGLA]
def create_dataset(path, num_examples):
    lines = io.open(path, encoding='UTF-8').read().strip().split('\n')
    
    """
    test test test
    """
    #print(lines)
    #print("\n")
    #print(num_examples)
    """
    test test test
    """

    question = []
    answer = []
    for l in lines[:num_examples]:
        #print(l)
        w = l.split('\t')
        """
        test test test
        """
        #print(w)
        #print("\n")
        """
        test test test
        """
        question.append(w[0])
        answer.append(w[1])
    vocabularyCounter(question, answer) 
    word_pairs = [[preprocess_sentence(w) for w in l.split('\t')]  for l in lines[:num_examples]]
    #print(word_pairs)
#     rearrange dataset src and target
    for indd in range(len(word_pairs)):
        temp = word_pairs[indd][0]
        word_pairs[indd][0] = word_pairs[indd][1]
        word_pairs[indd][1] = temp
    #print(word_pairs)
    return zip(*word_pairs)


In [11]:
# preprocessing function declear and vocabulary checking
bn, en = create_dataset(path_to_file, None)

#print("\n\n", en[4])
#print(bn)

Total English words: 877
Unique English words: 420
Most Common Words: "the" "is" "of" ""Which" "in" ""What" "are" ""Is" "with" "protein"

Total Bangla words: 2984
Unique Bangla words: 1454
Most Common Words: "of" "the" "and" "," "in" "to" "."" "(" ")" "a"


In [12]:
#function declearation for 
#     1. length of sentence
#https://stackoverflow.com/questions/51956000/what-does-keras-tokenizer-method-exactly-do
#https://stackoverflow.com/questions/61661160/comparison-of-tf-keras-preprocessing-text-tokenizer-and-tfds-features-text-tok
def max_length(tensor):
    return max(len(t) for t in tensor)

#     2. tokenizer
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')

  return tensor, lang_tokenizer
#     3. load dataset
def load_dataset(path, num_examples=None):
    # creating cleaned input, output pairs
    targ_lang, inp_lang  = create_dataset(path, num_examples)
    
    input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
    target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

    return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer
'''
   Total English words: 40
Unique English words: 9
Most Common Words: "Do" "lincRNAs" "play" "a" "role" "human" "cancer?" "1in" "in"

Total Bangla words: 66
Unique Bangla words: 50
Most Common Words: "lincRNAs" "of" "and" "to" "polyadenylation" "in" "cancer" "we" "the" "Genome-wide"
(array([[ 1,  2,  3,  4,  5,  6, 12,  7,  8,  9, 10],
        [ 1,  2,  3,  4,  5,  6, 11,  7,  8,  9, 10],
        [ 1,  2,  3,  4,  5,  6, 11,  7,  8,  9, 10],
        [ 1,  2,  3,  4,  5,  6, 11,  7,  8,  9, 10],
        [ 1,  2,  3,  4,  5,  6, 11,  7,  8,  9, 10]], dtype=int32),
 array([[ 1, 13, 14,  7, 15, 16,  5,  2,  8,  9, 10, 17,  3,  4,  0,  0,
          0,  0,  0,  0],
        [ 1, 18, 19, 11, 20,  2,  6, 21, 22,  9, 23, 24,  5, 25, 26,  3,
          4,  0,  0,  0],
        [ 1, 27, 28,  5,  2, 29, 30, 31, 32, 33, 34,  3,  4,  0,  0,  0,
          0,  0,  0,  0],
        [ 1,  6, 35, 12, 36,  5,  2,  8, 37,  6, 10, 38, 39, 40, 41,  3,
          4,  0,  0,  0],
        [ 1, 42, 11, 43, 44, 45, 46,  7, 47, 48,  6, 49, 12, 50, 51,  2,
          7, 52,  3,  4]], dtype=int32),
 <keras_preprocessing.text.Tokenizer at 0x7f9045bf34a8>,
 <keras_preprocessing.text.Tokenizer at 0x7f9045d264a8>)
 ''' 

'\n   Total English words: 40\nUnique English words: 9\nMost Common Words: "Do" "lincRNAs" "play" "a" "role" "human" "cancer?" "1in" "in"\n\nTotal Bangla words: 66\nUnique Bangla words: 50\nMost Common Words: "lincRNAs" "of" "and" "to" "polyadenylation" "in" "cancer" "we" "the" "Genome-wide"\n(array([[ 1,  2,  3,  4,  5,  6, 12,  7,  8,  9, 10],\n        [ 1,  2,  3,  4,  5,  6, 11,  7,  8,  9, 10],\n        [ 1,  2,  3,  4,  5,  6, 11,  7,  8,  9, 10],\n        [ 1,  2,  3,  4,  5,  6, 11,  7,  8,  9, 10],\n        [ 1,  2,  3,  4,  5,  6, 11,  7,  8,  9, 10]], dtype=int32),\n array([[ 1, 13, 14,  7, 15, 16,  5,  2,  8,  9, 10, 17,  3,  4,  0,  0,\n          0,  0,  0,  0],\n        [ 1, 18, 19, 11, 20,  2,  6, 21, 22,  9, 23, 24,  5, 25, 26,  3,\n          4,  0,  0,  0],\n        [ 1, 27, 28,  5,  2, 29, 30, 31, 32, 33, 34,  3,  4,  0,  0,  0,\n          0,  0,  0,  0],\n        [ 1,  6, 35, 12, 36,  5,  2,  8, 37,  6, 10, 38, 39, 40, 41,  3,\n          4,  0,  0,  0],\n        [ 1,

In [13]:
# Try experimenting with the size of that dataset
num_examples = 1000
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file, num_examples)


print(input_tensor)
print("\n")
print(inp_lang)


# Calculate max_length of the target tensors
max_length_targ, max_length_inp = max_length(target_tensor), max_length(input_tensor)

# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# Show length
print("Size:- Train_X-", len(input_tensor_train), "Train_Y-", len(target_tensor_train), "Test_X-", len(input_tensor_val), "Test_Y-", len(target_tensor_val))

Total English words: 877
Unique English words: 420
Most Common Words: "the" "is" "of" ""Which" "in" ""What" "are" ""Is" "with" "protein"

Total Bangla words: 2984
Unique Bangla words: 1454
Most Common Words: "of" "the" "and" "," "in" "to" "."" "(" ")" "a"
[[ 1 43 89 ...  0  0  0]
 [ 1  9  4 ...  0  0  0]
 [ 1  6 31 ...  0  0  0]
 ...
 [ 1  9  4 ...  0  0  0]
 [ 1  9  4 ...  0  0  0]
 [ 1  9  4 ...  0  0  0]]


Size:- Train_X- 80 Train_Y- 80 Test_X- 20 Test_Y- 20


In [14]:
# showing example of tokenization
def convert(lang, tensor):
  for t in tensor:
   # print("t t t",t)

    if t!=0:
      print ("%d ----> %s" % (t, lang.index_word[t]))
    
    
print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[0])

Input Language; index to word mapping
1 ----> <start>
6 ----> which
10 ----> are
5 ----> the
346 ----> yamanaka
88 ----> factors
3 ----> ?
2 ----> <end>

Target Language; index to word mapping
4 ----> <start>
18 ----> as
1 ----> the
61 ----> first
1222 ----> descriptions
2 ----> of
1223 ----> ipsc
1224 ----> production
13 ----> with
1 ----> the
1225 ----> transcription
96 ----> factors
1226 ----> sox2
6 ----> ,
1227 ----> klf4
6 ----> ,
1228 ----> oct4
3 ----> and
136 ----> c-myc
6 ----> ,
1229 ----> called
1 ----> the
1230 ----> yamanaka
96 ----> factors
6 ----> ,
10 ----> a
1231 ----> variety
2 ----> of
1232 ----> methods
38 ----> has
28 ----> been
68 ----> developed
9 ----> to
1233 ----> convert
223 ----> somatic
39 ----> cells
24 ----> from
191 ----> all
220 ----> germ
1234 ----> layers
9 ----> to
1235 ----> pluripotent
1236 ----> stem
39 ----> cells
8 ----> .
5 ----> <end>


In [15]:
#https://www.geeksforgeeks.org/tensorflow-tf-data-dataset-from_tensor_slices/
#https://stackoverflow.com/questions/53514495/what-does-batch-repeat-and-shuffle-do-with-tensorflow-dataset

BUFFER_SIZE = len(target_tensor_train)
#print(BUFFER_SIZE)
BATCH_SIZE = 4
print("\n")
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
#print(steps_per_epoch)

embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
print(dataset)

dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
print(dataset)

example_input_batch, example_target_batch = next(iter(dataset))
print("Input-output shape batch", example_input_batch.shape, example_target_batch.shape)




<ShuffleDataset shapes: ((19,), (76,)), types: (tf.int32, tf.int32)>
<BatchDataset shapes: ((4, 19), (4, 76)), types: (tf.int32, tf.int32)>
Input-output shape batch (4, 19) (4, 76)


In [16]:
print(input_tensor_train)

[[  1   6  10 ...   0   0   0]
 [  1   6  14 ...   0   0   0]
 [  1   6  81 ...   0   0   0]
 ...
 [  1 245 246 ...   0   0   0]
 [  1  21 201 ...   0   0   0]
 [  1   9  43 ...   0   0   0]]


In [17]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units, activation='linear',return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (4, 19, 1024)
Encoder Hidden state shape: (batch size, units) (4, 1024)


In [18]:
# Attention and context_vector
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    hidden_with_time_axis = tf.expand_dims(query, 1)
    score = self.V(tf.nn.sigmoid(self.W1(values) + self.W2(hidden_with_time_axis)))
    
    attention_weights = tf.nn.sigmoid(score)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) (4, 1024)
Attention weights shape: (batch_size, sequence_length, 1) (4, 19, 1)


In [19]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units, activation='tanh',return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    context_vector, attention_weights = self.attention(hidden, enc_output)

    x = self.embedding(x)

    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)
    
    output = tf.reshape(output, (-1, output.shape[2]))
    x = self.fc(output)

    return x, state, attention_weights


decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)
sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)), sample_hidden, sample_output)
print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab size) (4, 1399)


In [20]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [21]:
# Checkpoint for saving the object
checkpoint_dir = './training_checkpoints_saved'
checkpoint_prefix = os.path.join(checkpoint_dir, "checkpointSaved")
checkpoint = tf.train.Checkpoint(optimizer=optimizer, encoder=encoder, decoder=decoder)

In [22]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [23]:
# start learn processing
EPOCHS = 10
lossMatrix = []
for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
        print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1, batch, batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)
    lossMatrix.append(total_loss / steps_per_epoch)
    print('Epoch {} Loss {:.4f}'.format(epoch + 1, total_loss / steps_per_epoch))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 2.5972
Epoch 2 Batch 0 Loss 3.4296
Epoch 2 Loss 2.6056
Time taken for 1 epoch 196.6787507534027 sec

Epoch 3 Batch 0 Loss 2.1430
Epoch 4 Batch 0 Loss 2.0120
Epoch 4 Loss 2.3883
Time taken for 1 epoch 214.854896068573 sec

Epoch 5 Batch 0 Loss 1.8375
Epoch 6 Batch 0 Loss 1.5676
Epoch 6 Loss 2.0559
Time taken for 1 epoch 205.58678102493286 sec

Epoch 7 Batch 0 Loss 1.7578
Epoch 8 Batch 0 Loss 1.8896
Epoch 8 Loss 1.6017
Time taken for 1 epoch 211.89366269111633 sec

Epoch 9 Batch 0 Loss 1.5672
Epoch 10 Batch 0 Loss 1.2217
Epoch 10 Loss 1.1282
Time taken for 1 epoch 210.57285523414612 sec



In [24]:
def evaluate(sentence):
    attention_plot = np.zeros((max_length_targ, max_length_inp))

    sentence = preprocess_sentence(sentence)
 
    inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                           maxlen=max_length_inp,
                                                           padding='post')
    inputs = tf.convert_to_tensor(inputs)

    result = ''

    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)

        # storing the attention weights to plot later on
        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention_plot[t] = attention_weights.numpy()

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += targ_lang.index_word[predicted_id] + ' '

        if targ_lang.index_word[predicted_id] == '<end>':
            return result, sentence, attention_plot

        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence, attention_plot

# function for plotting the attention weights
def plot_attention(attention, sentence, predicted_sentence):
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(attention, cmap='viridis')

    fontdict = {'fontsize': 14}

    ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
    ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()
    
def translate(sentence):
    result, sentence, attention_plot = evaluate(sentence)

    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))

    attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
#     plot_attention(attention_plot, sentence.split(' '), result.split(' '))

In [25]:
# load checklist object from saving
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [42]:
translate(" what is cancer?")

Input: <start> what is cancer ? <end>
Predicted translation: the global care and c-myc , we show that the global care and c-myc , we show that the global care and c-myc , we show that the global care and c-myc , we show that the global care and c-myc , we show that the global care and c-myc , we show that the global care and c-myc , we show that the global care and c-myc , we show that the global care and 
